In [ ]:
import tweepy
import pandas as pd
import re
from textblob import TextBlob
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
auth = tweepy.OAuthHandler(api_key,api_secret_key)
auth.set_access_token(access_token,access_token_secret)
api = tweepy.API(auth)

In [ ]:
searchfile = api.search(q="upaya vaksinasi -filter:retweets",lang="id",count=250,tweet_mode="extended")

In [ ]:
for tweet in searchfile[0:5]:
    print(tweet.full_text + '\n')

In [ ]:
tweets = []
users = []
for tweet in searchfile:
    tweet_usr = tweet.full_text
    username = tweet.user.screen_name
    tweets.append(tweet_usr)
    users.append(username)
data_tweet = list(zip(users, tweets))

In [ ]:
#create Dataframe
df_content = pd.DataFrame(data_tweet , columns=['UserName','Tweet'])
df_content.head()

In [ ]:
def cleaning_txt(text):
    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())
    text = re.sub(r'RT[\s]+','',text)
    return text

In [ ]:
df_content['Tweet'] = df_content['Tweet'].apply(cleaning_txt)
df_content.drop_duplicates(subset='Tweet',inplace=True)

In [ ]:
df_content.head()

In [ ]:
def Polarity(text):
    analysis = TextBlob(text)
    try:
        analysis = analysis.translate(to="en")
    except Exception as e:
        print(e)
    return analysis.sentiment.polarity

In [ ]:
df_content['Polarity'] = df_content['Tweet'].apply(Polarity

In [ ]:

df_content.head()

In [ ]:
def analysis_classify(polarity_value):
    if polarity_value < 0:
        return 'Negative'
    elif polarity_value == 0:
        return 'Neutral'
    else:
        return 'Positive'

In [ ]:
df_content['Classify'] = df_content['Polarity'].apply(analysis_classify)

In [ ]:
df_content

In [ ]:
positive_tweet = df_content[df_content.Classify == 'Positive']
percent_positive = len(positive_tweet)/len(df_content) * 100
print('Percentage',percent_positive,'%')

In [ ]:
negative_tweet = df_content[df_content.Classify == 'Negative']
percent_negative = len(negative_tweet)/len(df_content) * 100
print('Percentage',percent_negative,'%')

In [ ]:
netral_tweet = df_content[df_content.Classify == 'Neutral']
percent_netral = len(netral_tweet)/len(df_content) * 100
print('Percentage',percent_netral,'%')

In [ ]:
df_content['Classify'].value_counts()

plt.title('Sentiment Analysis Vaksinasi Covid19 in Indonesia')
plt.xlabel('Sentiment')
plt.ylabel('Counts')
df_content['Classify'].value_counts().plot(kind='bar')
plt.show

In [ ]:
labels = df_content['Classify'].unique()
sizes = [percent_positive,percent_negative,percent_netral]
plt.pie(sizes,labels=labels,autopct='%1.1f%%',startangle=90,shadow=True)
plt.title('Percentage of Sentiment Analysis Vaksinasi Covid19 in Indonesia')
plt.axis('equal')
plt.show()